In [1]:
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [3]:
# STEP 1: Set up Gemini API key and import ADK components

import os
from kaggle_secrets import UserSecretsClient

# 1) Load your Gemini API key from Kaggle secrets
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key loaded from Kaggle secrets.")
except Exception as e:
    raise RuntimeError(
        "❌ Could not load GOOGLE_API_KEY from Kaggle secrets. "
        "Make sure you've added it in 'Settings' → 'Secrets'."
    ) from e

✅ Gemini API key loaded from Kaggle secrets.


In [18]:
# 2) Import Gemini + ADK core components (same style as course notebooks)

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK + Gemini imports successful.")

# 3) Create ADK Gemini model wrapper (required by Agent)

# Must use the exact model name supported by your API:
# From your list, this one exists:
# models/gemini-pro-latest

model = Gemini(model="models/gemini-2.5-flash")

print("✅ ADK Gemini model wrapper created.")

✅ ADK + Gemini imports successful.
✅ ADK Gemini model wrapper created.


In [9]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Correct model name for v1beta in Kaggle
model = genai.GenerativeModel("models/gemini-2.5-flash")

print("✅ Gemini 2.5 Flash initialized!")

✅ Gemini 2.5 Flash initialized!


In [10]:
class SummarizeTool:
    def run(self, text: str):
        prompt = f"Summarize the following text in 3–4 simple sentences:\n\n{text}"
        response = model.generate_content(prompt)
        return {"summary": response.text}

In [11]:
class SummarizerAgent:
    def __init__(self):
        self.tool = SummarizeTool()

    def handle(self, user_input: str):
        return self.tool.run(user_input)

In [12]:
agent = SummarizerAgent()

text = """
Kaggle’s 5-Day AI Agents Intensive teaches core concepts including 
agent architectures, tool creation, multi-agent design, memory, 
observability, and agent-to-agent communication using Google’s ADK.
"""

output = agent.handle(text)
output

{'summary': 'Kaggle offers a 5-Day AI Agents Intensive program. It teaches essential concepts for designing AI agents, such as architectures, tool creation, and multi-agent systems. The intensive also covers agent memory, observability, and inter-agent communication. These concepts are explored using Google’s ADK.'}

In [13]:
response = model.generate_content("Explain quantum computing in 2 sentences.")
print(response.text)

Quantum computing utilizes quantum-mechanical phenomena like superposition and entanglement to process information in fundamentally new ways compared to classical computers. This allows it to potentially solve certain complex problems intractable for conventional machines, offering advancements in fields like drug discovery, materials science, and cryptography.
